In [ ]:
pip install selenium webdriver_manager

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--log-level=3')  # 로깅 수준 조정
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(service=Service(
    ChromeDriverManager().install()), options=options)

# Sephora 웹사이트 접속
url = "https://www.sephora.com/brands-list"
driver.get(url)

# 페이지 로딩 대기
time.sleep(5)

# 팝업 메시지 닫기
try:
    popup_close_button = driver.find_element(By.XPATH, '//button[@class="css-1kna575"]')
    popup_close_button.click()
except:
    print("팝업 메시지를 찾을 수 없습니다.")

action = ActionChains(driver)

brand_names = []
brand_links = []  # 브랜드 링크를 저장할 리스트 추가
brand_item_num = [18, 23, 26, 24, 7, 14, 15, 12, 12, 12, 18, 14, 22, 14, 13, 16, 14, 33, 17, 1, 14, 6, 3, 5]

for i in range(0, 24):
    print(i)
    for j in range(1, brand_item_num[i]):
        element = driver.find_elements(By.XPATH, "/html/body/div[2]/div/main/div["+str(i+2)+"]/div[2]/ul/li["+str(j)+"]/a/span")
        brand_names.append(element[0].text)
        
        # href 속성 가져오기
        link_element = driver.find_element(By.XPATH, f"/html/body/div[2]/div/main/div[{i + 2}]/div[2]/ul/li[{j}]/a")
        brand_links.append(link_element.get_attribute('href'))
        
        if(j == (brand_item_num[i] - 1)):
            action.move_to_element(element[0]).perform()
driver.quit()

print(len(brand_names))
print(sum(brand_item_num) - len(brand_item_num))

print(len(brand_links))

In [ ]:
print(brand_names)

print(brand_links)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd

# 각 brand link에 들어가서 product name, image 가져오기 

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--log-level=3')  # 로깅 수준 조정
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(service=Service(
    ChromeDriverManager().install()), options=options)

df = pd.DataFrame(columns=['brand_name', 'product_name', 'image_url'])

problematic_brand = []

for idx, brand_link in enumerate(brand_links):
    
    driver.get(brand_link)
    
    # 페이지 로딩 대기
    time.sleep(5)

    # 팝업 메시지 닫기
    try:
        popup_close_button = driver.find_element(By.XPATH, '//button[@class="css-1kna575"]')
        popup_close_button.click()
    except:
        print("팝업 메시지를 찾을 수 없습니다.")
        
    # "Show More Products" 버튼이 있는 경우 클릭하고 버튼이 사라질 때까지 반복
    while True:
        try:
            show_more_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "button.css-bk5oor.e65zztl0")))
            show_more_button.click()
            time.sleep(5)  # 페이지 로드 대기
            
            # 팝업 메시지 닫기
            try:
                popup_close_button = driver.find_element(By.XPATH, '//button[@class="css-1kna575"]')
                popup_close_button.click()
            except:
                print("팝업 메시지를 찾을 수 없습니다.")
            
            
        except:
            print("No more Show More Products button found or not clickable")
            break

    # 맨 위로 이동
    driver.execute_script("window.scrollTo(0, 0);")
    
    # 요소 개수 구하기
    elements = driver.find_elements(By.XPATH, "/html/body/div[2]/div/div/div/div[2]/main/div/div[2]/div")
    element_count = len(elements) - 2  # 첫 번째 요소, 마지막 요소는 필요없으므로 -2

    print(element_count)
    if(element_count == -2):
        problematic_brand.append(brand_names[idx])
    
    for i in range(2, element_count + 2):  # 2에서 시작
        
        try:
            # 이미지 요소 가져오기
            img_element = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f"/html/body/div[2]/div/div/div/div[2]/main/div/div[2]/div[{i}]/a/div[1]/div/picture/img")))
            driver.execute_script("arguments[0].scrollIntoView();", img_element)

            # 텍스트 요소 가져오기
            text_element = driver.find_element(By.XPATH, f"/html/body/div[2]/div/div/div/div[2]/main/div/div[2]/div[{i}]/a/span[2]")

            new_data = {'brand_name': brand_names[idx], 'product_name': text_element.text, 'image_url': img_element.get_property("src")}

            # 컬럼 이름을 리스트 형태로 만들어서 데이터프레임에 추가
            df = pd.concat([df, pd.DataFrame([new_data], columns=df.columns)], ignore_index=True)
        except:
            problematic_brand.append(brand_names[idx])
            break

driver.quit()

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd

# 각 problemetic link에서 product name, image 가져오기
# 지금까지의 문제 유형
# 1, for i in range(2, element_count + 2) 에서 2가 아닌 3부터 시작해야 하는 경우
# 2. 너무 개수가 많아 driver.execute_script("window.scrollTo(0, 0);") 을 의도적으로 해줘야 하는 경우

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--log-level=3')  # 로깅 수준 조정
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(service=Service(
    ChromeDriverManager().install()), options=options)

temp_df = pd.DataFrame(columns=['brand_name', 'product_name', 'image_url'])

link = "https://www.sephora.com/brand/shiseido"
driver.get(link)
    
# 페이지 로딩 대기
time.sleep(5)

# 팝업 메시지 닫기
try:
    popup_close_button = driver.find_element(By.XPATH, '//button[@class="css-1kna575"]')
    popup_close_button.click()
except:
    print("팝업 메시지를 찾을 수 없습니다.")

# "Show More Products" 버튼이 있는 경우 클릭하고 버튼이 사라질 때까지 반복
while True:
    try:
        show_more_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button.css-bk5oor.e65zztl0")))
        show_more_button.click()
        time.sleep(5)  # 페이지 로드 대기

        # 팝업 메시지 닫기
        try:
            popup_close_button = driver.find_element(By.XPATH, '//button[@class="css-1kna575"]')
            popup_close_button.click()
        except:
            print("팝업 메시지를 찾을 수 없습니다.")


    except:
        print("No more Show More Products button found or not clickable")
        break

# 맨 위로 이동
driver.execute_script("window.scrollTo(0, 0);")

# 요소 개수 구하기
elements = driver.find_elements(By.XPATH, "/html/body/div[2]/div/div/div/div[2]/main/div/div[2]/div")
element_count = len(elements) - 2  # 첫 번째 요소, 마지막 요소는 필요없으므로 -2

print(element_count)
if(element_count == -2):
    problematic_brand.append(brand_names[idx])

for i in range(3, element_count + 2):  # 2에서 시작

    # 이미지 요소 가져오기
    img_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, f"/html/body/div[2]/div/div/div/div[2]/main/div/div[2]/div[{i}]/a/div[1]/div/picture/img")))
    driver.execute_script("arguments[0].scrollIntoView();", img_element)

    # 텍스트 요소 가져오기
    text_element = driver.find_element(By.XPATH, f"/html/body/div[2]/div/div/div/div[2]/main/div/div[2]/div[{i}]/a/span[2]")

    new_data = {'brand_name': 'Shiseido', 'product_name': text_element.text, 'image_url': img_element.get_property("src")}

    # 컬럼 이름을 리스트 형태로 만들어서 데이터프레임에 추가
    temp_df = pd.concat([temp_df, pd.DataFrame([new_data], columns=df.columns)], ignore_index=True)
    

driver.quit()

In [ ]:
temp_df

In [ ]:
df = pd.concat([df, temp_df], ignore_index=True)

In [ ]:
df.to_csv('/workspace/web_scraping_selenium/cosmetic_info.csv', index=False, encoding='utf-8-sig')